In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

## Discriminator

In [2]:
def define_discriminator(in_shape=(28, 28, 1)):
  # weigth intialization
  init = keras.initializers.RandomNormal(stddev=0.02)
  model = keras.Sequential()
  # downsample to 14x14
  model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init, input_shape=in_shape))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(alpha=0.2))
  # downsample to 7x7
  model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(alpha=0.2))
  # classifier
  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation="linear", kernel_initializer=init))
  # compile model with L2 loss
  model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))
  return model

discriminator = define_discriminator()
discriminator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        1088      
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 64)       256       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         131200    
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 128)        512       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 7, 128)         0

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 1)                 6273      
                                                                 
Total params: 139,329
Trainable params: 138,945
Non-trainable params: 384
_________________________________________________________________


## Generator

In [3]:
# define the standalone generator model 
def define_generator(latent_dim): 
  # weight initialization 
  init = keras.initializers.RandomNormal(stddev=0.02) 
  # define model 
  model = keras.Sequential() 
  # foundation for 7x7 image 
  n_nodes = 256 * 7 * 7 
  model.add(layers.Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim)) 
  model.add(layers.BatchNormalization()) 
  model.add(layers.Activation('relu')) 
  model.add(layers.Reshape((7, 7, 256)))
   # upsample to 14x14
  model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)) 
  model.add(layers.BatchNormalization()) 
  model.add(layers.Activation('relu')) 
  # upsample to 28x28
  model.add(layers.Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)) 
  model.add(layers.BatchNormalization()) 
  model.add(layers.Activation('relu')) 
  # output 28x28x1 
  model.add(layers.Conv2D(1, (7,7), padding='same', kernel_initializer=init)) 
  model.add(layers.Activation('tanh'))
  return model

generator = define_generator(100)
generator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 12544)             1266944   
                                                                 
 batch_normalization_2 (Batc  (None, 12544)            50176     
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 12544)             0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 128)      524416    
 nspose)                                                         
                                                                 
 batch_normalization_3 (Batc  (None, 14, 14, 128)     

## GAN

In [4]:
def define_gan(generator, discriminator):
  discriminator.trainable = False
  model = keras.Sequential()
  model.add(generator)
  model.add(discriminator)
  model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))
  return model

## Dataset and Evaluation Functions

In [5]:
def load_real_samples():
  (x_train, _), (_, _) = keras.datasets.mnist.load_data()
  x = np.expand_dims(x_train, axis=-1)
  x = x.astype('float32')
  x = (x -  127.5) / 127.5
  return x

def generate_real_samples(dataset, n_samples):
  ix = np.random.randint(0, dataset.shape[0], n_samples)
  x = dataset[ix]
  y = np.ones((n_samples, 1))
  return x, y

def generate_latent_points(latent_dim, n_samples):
  x_input = np.random.randn(latent_dim * n_samples)
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

def generate_fake_samples(generator, latent_dim, n_samples):
  x_input = generate_latent_points(latent_dim, n_samples)
  x = generator.predict(x_input)
  y = np.zeros((n_samples, 1))
  return x, y

def summarize_performance(step, g_model, latent_dim, n_samples=100):
  x, _ = generate_fake_samples(g_model, latent_dim, n_samples)
  x = (x + 1) / 2.0
  for i in range(10 * 10):
    plt.subplot(10, 10, 1 + i)
    plt.axis("off")
    plt.imshow(x[i, :, :, 0], cmap='gray_r')
  filename1 = 'generator_model_%03d.png' % (step+1)
  plt.savefig(filename1)
  plt.close()
  filename2 = 'model_%06d.h5' % (step+1)
  g_model.save(filename2)
  print('Saved %s and %s' % (filename1, filename2))

def plot_history(d1_hist, d2_hist, g_hist):
  plt.plot(d1_hist, label='dloss1') 
  plt.plot(d2_hist, label='dloss2')
  plt.plot(g_hist, label='gloss')
  plt.legend()
  filename = 'plot_line_plot_loss.png'
  plt.savefig(filename)
  plt.close()
  print('Saved %s' % (filename))

## Train

In [6]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=64):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  n_steps = bat_per_epo * n_epochs
  half_batch = int(n_batch / 2)
  d1_hist, d2_hist, g_hist = list(), list(), list()
  for i in range(n_steps):
    x_real, y_real = generate_real_samples(dataset, half_batch)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
    d_loss1 = d_model.train_on_batch(x_real, y_real)
    d_loss2 = d_model.train_on_batch(x_fake, y_fake)
    z_input = generate_latent_points(latent_dim, n_batch)
    y_real2 = np.ones((n_batch, 1))
    g_loss = gan_model.train_on_batch(z_input, y_real2)
    # summarize loss on this batch
    print('>%d, d1=%.3f, d2=%.3f, g=%.3f' % (i+1, d_loss1, d_loss2, g_loss))
    # record history
    d1_hist.append(d_loss1)
    d2_hist.append(d_loss2)
    g_hist.append(g_loss) 
    # evaluate the model performance every 'epoch'
    if (i+1) % (bat_per_epo * 1) == 0:
      summarize_performance(i, g_model, latent_dim)
  plot_history(d1_hist, d2_hist, g_hist)

latent_dim = 100
discriminator = define_discriminator()
generator = define_generator(latent_dim)
gan_model = define_gan(generator, discriminator)
dataset = load_real_samples()
print(dataset.shape)
train(generator, discriminator, gan_model, dataset, latent_dim)

11490434/11490434 [==============================] - 1s 0us/step
(60000, 28, 28, 1)
1/1 [==============================] - 10s 10s/step
>1, d1=1.253, d2=0.180, g=0.991
1/1 [==============================] - 0s 17ms/step
>2, d1=4.279, d2=0.478, g=0.903
1/1 [==============================] - 0s 17ms/step
>3, d1=0.899, d2=1.295, g=0.892
1/1 [==============================] - 0s 17ms/step
>4, d1=0.987, d2=0.229, g=0.903
1/1 [==============================] - 0s 19ms/step
>5, d1=0.469, d2=0.172, g=0.876
1/1 [==============================] - 0s 20ms/step
>6, d1=0.408, d2=0.377, g=0.874
1/1 [==============================] - 0s 18ms/step
>7, d1=0.409, d2=0.166, g=0.850
1/1 [==============================] - 0s 17ms/step
>8, d1=0.501, d2=0.507, g=0.882
1/1 [==============================] - 0s 19ms/step
>9, d1=0.660, d2=0.424, g=0.844
1/1 [==============================] - 0s 19ms/step
>10, d1=0.763, d2=1.107, g=0.901
1/1 [==============================] - 0s 34ms/step
>11, d1=0.889, d2=2.324

Saved generator_model_937.png and model_000937.h5
1/1 [==============================] - 0s 21ms/step
>938, d1=0.039, d2=0.001, g=0.000
1/1 [==============================] - 0s 26ms/step
>939, d1=0.029, d2=0.000, g=0.001
1/1 [==============================] - 0s 23ms/step
>940, d1=0.023, d2=0.000, g=0.000
1/1 [==============================] - 0s 47ms/step
>941, d1=0.023, d2=0.000, g=0.000
1/1 [==============================] - 0s 88ms/step
>942, d1=0.021, d2=0.000, g=0.000
1/1 [==============================] - 0s 88ms/step
>943, d1=0.022, d2=0.000, g=0.000
1/1 [==============================] - 0s 59ms/step
>944, d1=0.037, d2=0.005, g=0.001
1/1 [==============================] - 0s 81ms/step
>945, d1=0.026, d2=0.010, g=0.002
1/1 [==============================] - 0s 91ms/step
>946, d1=0.024, d2=0.016, g=0.000
1/1 [==============================] - 0s 95ms/step
>947, d1=0.032, d2=0.025, g=0.002
1/1 [==============================] - 0s 86ms/step
>948, d1=0.020, d2=0.033, g=0.000
1/1 

Saved generator_model_1874.png and model_001874.h5
1/1 [==============================] - 0s 33ms/step
>1875, d1=0.019, d2=0.003, g=0.004
1/1 [==============================] - 0s 92ms/step
>1876, d1=0.012, d2=0.004, g=0.004
1/1 [==============================] - 0s 56ms/step
>1877, d1=0.019, d2=0.010, g=0.002
1/1 [==============================] - 0s 62ms/step
>1878, d1=0.014, d2=0.018, g=0.006
1/1 [==============================] - 0s 35ms/step
>1879, d1=0.013, d2=0.025, g=0.000
1/1 [==============================] - 0s 88ms/step
>1880, d1=0.010, d2=0.032, g=0.011
1/1 [==============================] - 0s 51ms/step
>1881, d1=0.011, d2=0.041, g=0.000
1/1 [==============================] - 0s 56ms/step
>1882, d1=0.010, d2=0.053, g=0.017
1/1 [==============================] - 0s 53ms/step
>1883, d1=0.026, d2=0.053, g=0.001
1/1 [==============================] - 0s 73ms/step
>1884, d1=0.052, d2=0.042, g=0.024
1/1 [==============================] - 0s 51ms/step
>1885, d1=0.083, d2=0.024, 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-1b94a1cdbd8b>", line 31, in <cell line: 31>
    train(generator, discriminator, gan_model, dataset, latent_dim)
  File "<ipython-input-6-1b94a1cdbd8b>", line 8, in train
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
  File "<ipython-input-5-d08c9869d663>", line 21, in generate_fake_samples
    x = generator.predict(x_input)
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 2349, in predict
    data_handler = data_adapter.get_data_handler(
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/data_adapter.py", line 1583, in get_data_handler
    return DataHandler(*args,

TypeError: ignored

## Generate Images With LSGAN

In [ ]:
def generate_latent_points(latent_dim, n_samples):
  x_input = np.random.randn(latent_dim * n_samples)
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

def plot_generated(examples, n):
  for i in range(n * n):
    plt.subplot(n, n, 1 + i)
    plt.axis('off')
    plt.imshow(examples[i, :, :, 0], cmap='gray_r')
  plt.show()

model = keras.models.load_model('model_018740.h5')
latent_points = generate_latent_points(100, 100)
x = model.predict(latent_points)
plot_generated(x, 10)